In [ ]:
import './../../loadenv.mjs'

In [2]:
import { getModel } from '../../utils.mjs'
import { TavilySearchResults } from '@langchain/community/tools/tavily_search'
import { HumanMessage } from '@langchain/core/messages'
import { ToolNode } from '@langchain/langgraph/prebuilt'
import { StateGraph, MessagesAnnotation } from '@langchain/langgraph'

const tools = [
    new TavilySearchResults({ maxResults: 3 }),
]
const toolNode = new ToolNode(tools)

const model = getModel().bindTools(tools)

function shouldContinue({ messages }: typeof MessagesAnnotation.State) {
    const lastMessage = messages[messages.length - 1]

    if (lastMessage.additional_kwargs.tool_calls) {
        return 'tools'
    }
    return '__end__'
}

async function callModel(state: typeof MessagesAnnotation.State) {
    const response = await model.invoke(state.messages)

    return { messages: [response] }
}

const workflow = new StateGraph(MessagesAnnotation)
    .addNode('agent', callModel)
    .addEdge('__start__', 'agent')
    .addNode('tools', toolNode)
    .addEdge('tools', 'agent')
    .addConditionalEdges('agent', shouldContinue)

const app = workflow.compile()


In [ ]:
import { printGraph } from './../../utils.mjs'
await printGraph(app.getGraph())

In [ ]:
// Use the agent
const finalState = await app.invoke({
    messages: [new HumanMessage("what is the weather in sf")],
});
console.log(finalState.messages[finalState.messages.length - 1].content);

const nextState = await app.invoke({
    // Including the messages from the previous run gives the LLM context.
    // This way it knows we're asking about the weather in NY
    messages: [...finalState.messages, new HumanMessage("what about ny")],
});
console.log(nextState.messages[nextState.messages.length - 1].content);